In [1]:
import pandas as pd #dataframes
import matplotlib.pyplot as plt 
import numpy as np # n-dim object support
# do ploting inline instead of in a separate window
%matplotlib inline
import random

In [2]:
df_school = pd.read_csv("../Scraper/school_records.csv")
df_ps_game = pd.read_csv("../Scraper/post_season_game_records.csv")

In [3]:
df_school.shape

(3125, 21)

In [4]:
df_ps_game.shape

(441, 7)

In [5]:
df_school.head(5)

,year,team_name,fg_pg,ft_pg,three_pt_pg,orb_pg,drb_pg,ast_pg,stl_pg,blk_pg,...,pf_pg,pt_pg,opnt_pt_pg,fg_pct,three_p_pct,ft_pct,wl_pct,conf_wl_pct,srs,sos
0,2010.0,Air Force,20.387097,10.741935,5.677419,7.096774,27.222685,12.548387,5.000000,1.645161,...,17.645161,57.193548,63.129032,0.443,0.313,0.635,0.323,0.062500,-4.90,3.13
1,2010.0,Akron,25.057143,13.800000,6.714286,13.342857,35.875918,13.514286,6.085714,3.257143,...,19.485714,70.628571,65.514286,0.433,0.339,0.657,0.686,0.750000,2.82,-1.50
2,2010.0,Alabama A&M,22.185185,17.481481,4.000000,13.925926,36.669410,10.666667,9.222222,5.296296,...,20.370370,65.851852,69.666667,0.382,0.291,0.635,0.407,0.444444,-20.19,-13.71
3,2010.0,Alabama-Birmingham,22.441176,16.852941,5.205882,12.352941,36.342561,11.470588,6.558824,2.676471,...,17.970588,66.941176,60.382353,0.422,0.311,0.694,0.735,0.687500,9.46,2.90
4,2010.0,Alabama State,21.516129,15.290323,6.129032,12.903226,35.099896,12.903226,7.354839,4.161290,...,20.451613,64.451613,65.903226,0.404,0.324,0.641,0.516,0.666667,-14.41,-12.02


In [6]:
df_ps_game.head(5)

,year,team_1_name,team_1_score,team_1_seed,team_2_name,team_2_score,team_2_seed
0,2011,UTSA,46.0,16.0,Ohio State,75.0,1.0
1,2011,George Mason,61.0,8.0,Villanova,57.0,9.0
2,2011,Clemson,76.0,12.0,West Virginia,84.0,5.0
3,2011,Princeton,57.0,13.0,Kentucky,59.0,4.0
4,2011,Marquette,66.0,11.0,Xavier,55.0,6.0


In [7]:
df_school.isnull().values.any()

False

In [8]:
df_ps_game.isnull().values.any()

False

In [9]:
def resolve_team_name(team_name):
    #Apply hard-coded corrections to team names
    team_name_dict = {'Colorado-Colorado Springs':'Colorado',
                     'Colorado College': 'Colorado',
                     'UNC':'North Carolina',
                     'UConn':'Connecticut',
                     'LIU-Brooklyn':'Long Island University',
                     'UTSA':'Texas-San Antonio',
                     'Pitt':'Pittsburgh',
                     'BYU':'Brigham Young',
                     "St. Peter's": "Saint Peter's",
                     'VCU':'Virginia Commonwealth',
                     'Southern Miss':'Southern Mississippi',
                     'Detroit': 'Detroit Mercy',
                     'UNLV':'Nevada-Las Vegas',
                     'Ole Miss':'Mississippi',
                     "St. Joseph's":"Saint Joseph's",
                     'UCSB':'UC-Santa Barbara',
                     'SMU': 'Southern Methodist',
                     'USC':'South Carolina',
                     'LSU':'Louisiana State',
                     'UMass':'Massachusetts',
                     'ETSU':'East Tennessee State'}
    # TODO: for V2 add more corrections to the team_name_dict
    if(team_name in team_name_dict):
        return team_name_dict[team_name]
    return team_name
def get_school_stats(year, team_name):
    return df_school[(df_school['year'] == year) & (df_school['team_name'] == team_name)]
def get_vals(t_stats_list, key):
    ret = []
    for t_stat in t_stats_list:
        ret.append(t_stat[key].squeeze())
    return ret
def get_team_stats_dict_with_t1_win(t1_stats, t2_stats, t1_wins):
    return {'team_name_1':get_vals(t1_stats,'team_name'),'fg_pg_1':get_vals(t1_stats,'fg_pg'),'ft_pg_1':get_vals(t1_stats,'ft_pg'),
            'three_pt_pg_1':get_vals(t1_stats,'three_pt_pg'),'orb_pg_1':get_vals(t1_stats,'orb_pg'),'drb_pg_1':get_vals(t1_stats,'drb_pg'),
            'ast_pg_1':get_vals(t1_stats,'ast_pg'),'stl_pg_1':get_vals(t1_stats,'stl_pg'),'blk_pg_1':get_vals(t1_stats,'blk_pg'),
            'tov_pg_1':get_vals(t1_stats,'tov_pg'),'pf_pg_1':get_vals(t1_stats,'pf_pg'), 'pt_pg_1':get_vals(t1_stats,'pt_pg'),
            'opnt_pt_pg_1':get_vals(t1_stats,'opnt_pt_pg'),'fg_pct_1':get_vals(t1_stats,'fg_pct'),'three_p_pct_1':get_vals(t1_stats,'three_p_pct'),
            'ft_pct_1':get_vals(t1_stats,'ft_pct'),'wl_pct_1':get_vals(t1_stats,'wl_pct'),'conf_wl_pct_1':get_vals(t1_stats,'conf_wl_pct'),
            'srs_1':get_vals(t1_stats,'srs'),'sos_1':get_vals(t1_stats,'sos'),
            'team_name_2':get_vals(t2_stats,'team_name'),'fg_pg_2':get_vals(t2_stats,'fg_pg'),'ft_pg_2':get_vals(t2_stats,'ft_pg'),
            'three_pt_pg_2':get_vals(t2_stats,'three_pt_pg'),'orb_pg_2':get_vals(t2_stats,'orb_pg'),'drb_pg_2':get_vals(t2_stats,'drb_pg'),
            'ast_pg_2':get_vals(t2_stats,'ast_pg'),'stl_pg_2':get_vals(t2_stats,'stl_pg'),'blk_pg_2':get_vals(t2_stats,'blk_pg'),
            'tov_pg_2':get_vals(t2_stats,'tov_pg'),'pf_pg_2':get_vals(t2_stats,'pf_pg'), 'pt_pg_2':get_vals(t2_stats,'pt_pg'),
            'opnt_pt_pg_2':get_vals(t2_stats,'opnt_pt_pg'),'fg_pct_2':get_vals(t2_stats,'fg_pct'),'three_p_pct_2':get_vals(t2_stats,'three_p_pct'),
            'ft_pct_2':get_vals(t2_stats,'ft_pct'),'wl_pct_2':get_vals(t2_stats,'wl_pct'),'conf_wl_pct_2':get_vals(t2_stats,'conf_wl_pct'),
            'srs_2':get_vals(t2_stats,'srs'),'sos_2':get_vals(t2_stats,'sos'),
            't1_win':t1_wins}
def get_team_stats_dict(t1_stats, t2_stats):
    return {'team_name_1':get_vals(t1_stats,'team_name'),'fg_pg_1':get_vals(t1_stats,'fg_pg'),'ft_pg_1':get_vals(t1_stats,'ft_pg'),
            'three_pt_pg_1':get_vals(t1_stats,'three_pt_pg'),'orb_pg_1':get_vals(t1_stats,'orb_pg'),'drb_pg_1':get_vals(t1_stats,'drb_pg'),
            'ast_pg_1':get_vals(t1_stats,'ast_pg'),'stl_pg_1':get_vals(t1_stats,'stl_pg'),'blk_pg_1':get_vals(t1_stats,'blk_pg'),
            'tov_pg_1':get_vals(t1_stats,'tov_pg'),'pf_pg_1':get_vals(t1_stats,'pf_pg'), 'pt_pg_1':get_vals(t1_stats,'pt_pg'),
            'opnt_pt_pg_1':get_vals(t1_stats,'opnt_pt_pg'),'fg_pct_1':get_vals(t1_stats,'fg_pct'),'three_p_pct_1':get_vals(t1_stats,'three_p_pct'),
            'ft_pct_1':get_vals(t1_stats,'ft_pct'),'wl_pct_1':get_vals(t1_stats,'wl_pct'),'conf_wl_pct_1':get_vals(t1_stats,'conf_wl_pct'),
            'srs_1':get_vals(t1_stats,'srs'),'sos_1':get_vals(t1_stats,'sos'),
            'team_name_2':get_vals(t2_stats,'team_name'),'fg_pg_2':get_vals(t2_stats,'fg_pg'),'ft_pg_2':get_vals(t2_stats,'ft_pg'),
            'three_pt_pg_2':get_vals(t2_stats,'three_pt_pg'),'orb_pg_2':get_vals(t2_stats,'orb_pg'),'drb_pg_2':get_vals(t2_stats,'drb_pg'),
            'ast_pg_2':get_vals(t2_stats,'ast_pg'),'stl_pg_2':get_vals(t2_stats,'stl_pg'),'blk_pg_2':get_vals(t2_stats,'blk_pg'),
            'tov_pg_2':get_vals(t2_stats,'tov_pg'),'pf_pg_2':get_vals(t2_stats,'pf_pg'), 'pt_pg_2':get_vals(t2_stats,'pt_pg'),
            'opnt_pt_pg_2':get_vals(t2_stats,'opnt_pt_pg'),'fg_pct_2':get_vals(t2_stats,'fg_pct'),'three_p_pct_2':get_vals(t2_stats,'three_p_pct'),
            'ft_pct_2':get_vals(t2_stats,'ft_pct'),'wl_pct_2':get_vals(t2_stats,'wl_pct'),'conf_wl_pct_2':get_vals(t2_stats,'conf_wl_pct'),
            'srs_2':get_vals(t2_stats,'srs'),'sos_2':get_vals(t2_stats,'sos')}
def get_team_stats_dict_ps(t1_stats, t2_stats, t1_seeds, t2_seeds):
    return {'team_name_1':get_vals(t1_stats,'team_name'),'fg_pg_1':get_vals(t1_stats,'fg_pg'),'ft_pg_1':get_vals(t1_stats,'ft_pg'),
            'three_pt_pg_1':get_vals(t1_stats,'three_pt_pg'),'orb_pg_1':get_vals(t1_stats,'orb_pg'),'drb_pg_1':get_vals(t1_stats,'drb_pg'),
            'ast_pg_1':get_vals(t1_stats,'ast_pg'),'stl_pg_1':get_vals(t1_stats,'stl_pg'),'blk_pg_1':get_vals(t1_stats,'blk_pg'),
            'tov_pg_1':get_vals(t1_stats,'tov_pg'),'pf_pg_1':get_vals(t1_stats,'pf_pg'), 'pt_pg_1':get_vals(t1_stats,'pt_pg'),
            'opnt_pt_pg_1':get_vals(t1_stats,'opnt_pt_pg'),'fg_pct_1':get_vals(t1_stats,'fg_pct'),'three_p_pct_1':get_vals(t1_stats,'three_p_pct'),
            'ft_pct_1':get_vals(t1_stats,'ft_pct'),'wl_pct_1':get_vals(t1_stats,'wl_pct'),'conf_wl_pct_1':get_vals(t1_stats,'conf_wl_pct'),
            'srs_1':get_vals(t1_stats,'srs'),'sos_1':get_vals(t1_stats,'sos'),
            'team_name_2':get_vals(t2_stats,'team_name'),'fg_pg_2':get_vals(t2_stats,'fg_pg'),'ft_pg_2':get_vals(t2_stats,'ft_pg'),
            'three_pt_pg_2':get_vals(t2_stats,'three_pt_pg'),'orb_pg_2':get_vals(t2_stats,'orb_pg'),'drb_pg_2':get_vals(t2_stats,'drb_pg'),
            'ast_pg_2':get_vals(t2_stats,'ast_pg'),'stl_pg_2':get_vals(t2_stats,'stl_pg'),'blk_pg_2':get_vals(t2_stats,'blk_pg'),
            'tov_pg_2':get_vals(t2_stats,'tov_pg'),'pf_pg_2':get_vals(t2_stats,'pf_pg'), 'pt_pg_2':get_vals(t2_stats,'pt_pg'),
            'opnt_pt_pg_2':get_vals(t2_stats,'opnt_pt_pg'),'fg_pct_2':get_vals(t2_stats,'fg_pct'),'three_p_pct_2':get_vals(t2_stats,'three_p_pct'),
            'ft_pct_2':get_vals(t2_stats,'ft_pct'),'wl_pct_2':get_vals(t2_stats,'wl_pct'),'conf_wl_pct_2':get_vals(t2_stats,'conf_wl_pct'),
            'srs_2':get_vals(t2_stats,'srs'),'sos_2':get_vals(t2_stats,'sos'), 'team_1_seed':t1_seeds,
            'team_2_seed':t2_seeds}
def create_team_stats_df_w_t1_win(indeces_w_stats, t1_stats_list, t2_stats_list,t1_wins):
    # Adds column for wether team 1 wins or not
    # Assumes all lists are of the same length
    return pd.DataFrame(get_team_stats_dict_with_t1_win(t1_stats_list, t2_stats_list,t1_wins), index = indeces_w_stats)
def create_team_stats_df(indeces_w_stats, t1_stats_list, t2_stats_list):
    # Assumes all lists are of the same length
    return pd.DataFrame(get_team_stats_dict(t1_stats_list, t2_stats_list), index = indeces_w_stats)
def create_team_stats_df_ps(indeces_w_stats, t1_stats_list, t2_stats_list, t1_seeds, t2_seeds):
    # Only uses post season stats => inclu
    # Assumes all lists are of the same length
    return pd.DataFrame(get_team_stats_dict_ps(t1_stats_list, t2_stats_list, t1_seeds, t2_seeds), index = indeces_w_stats)
def get_team_stats_df(game_df, should_print=False):
    indeces_w_stats = []
    t1_stats_list = []
    t2_stats_list = []
    t1_wins_list = []
    for index, row in game_df.iterrows():
        year = row['year']
        team_1 = row['team_1_name']
        team_2 = row['team_2_name']
        team_1_score = row['team_1_score']
        team_2_score = row['team_2_score']
        t1_stats = get_school_stats(year, resolve_team_name(team_1))
        t2_stats = get_school_stats(year, resolve_team_name(team_2))

        if(len(t1_stats) > 0 and len(t2_stats) > 0):  
            indeces_w_stats.append(index)
            t1_stats_list.append(t1_stats)
            t2_stats_list.append(t2_stats)
            t1_wins_list.append(team_1_score > team_2_score)
        else:         
            if(should_print):
                print(year)
                if(len(t1_stats) < 1):
                    print(team_1)
                if(len(t2_stats) < 1):
                    print(team_2)
            
    print(len(indeces_w_stats))
    team_stats_df = create_team_stats_df_w_t1_win(indeces_w_stats, t1_stats_list, t2_stats_list, t1_wins_list)
    return team_stats_df
        

In [10]:
ps_team_stats_df = get_team_stats_df(df_ps_game, True)

441


In [11]:
ps_game_w_team_stats = pd.concat([df_ps_game, ps_team_stats_df], axis=1, join='inner')

In [12]:
ps_game_w_team_stats.head(3)

,year,team_1_name,team_1_score,team_1_seed,team_2_name,team_2_score,team_2_seed,team_name_1,fg_pg_1,ft_pg_1,...,pt_pg_2,opnt_pt_pg_2,fg_pct_2,three_p_pct_2,ft_pct_2,wl_pct_2,conf_wl_pct_2,srs_2,sos_2,t1_win
0,2011,UTSA,46.0,16.0,Ohio State,75.0,1.0,Texas-San Antonio,23.588235,16.058824,...,77.135135,59.675676,0.494,0.423,0.701,0.919,0.888889,25.84,8.38,False
1,2011,George Mason,61.0,8.0,Villanova,57.0,9.0,George Mason,25.764706,14.558824,...,72.242424,65.424242,0.438,0.348,0.757,0.636,0.500000,15.05,8.23,True
2,2011,Clemson,76.0,12.0,West Virginia,84.0,5.0,Clemson,23.823529,14.500000,...,69.787879,64.666667,0.429,0.337,0.711,0.636,0.611111,16.15,11.03,False


In [13]:
ps_game_w_team_stats.shape

(441, 48)

## Check team 1 winning true/false ratio

In [14]:
t1_win_map = {True:1, False:0}
ps_game_w_team_stats['t1_win'] = ps_game_w_team_stats['t1_win'].map(t1_win_map)
num_true = len(ps_game_w_team_stats.loc[ps_game_w_team_stats['t1_win'] == True])
num_false = len(ps_game_w_team_stats.loc[ps_game_w_team_stats['t1_win'] == False])
print("Number of True cases: {0} ({1:2.2f}%)".format(num_true, (num_true/(num_true+num_false))*100))
print("Number of False cases: {0} ({1:2.2f}%)".format(num_false, (num_false/(num_true+num_false))*100))

Number of True cases: 231 (52.38%)
Number of False cases: 210 (47.62%)


In [15]:
import sklearn.model_selection
ps_feature_col_names = ['team_1_seed', 'team_2_seed','fg_pg_1','ft_pg_1',
            'three_pt_pg_1','orb_pg_1','drb_pg_1',
            'ast_pg_1','stl_pg_1','blk_pg_1',
            'tov_pg_1','pf_pg_1', 'pt_pg_1',
            'opnt_pt_pg_1','fg_pct_1','three_p_pct_1',
            'ft_pct_1','wl_pct_1','conf_wl_pct_1',
            'srs_1','sos_1',
            'fg_pg_2','ft_pg_2',
            'three_pt_pg_2','orb_pg_2','drb_pg_2',
            'ast_pg_2','stl_pg_2','blk_pg_2',
            'tov_pg_2','pf_pg_2', 'pt_pg_2',
            'opnt_pt_pg_2','fg_pct_2','three_p_pct_2',
            'ft_pct_2','wl_pct_2','conf_wl_pct_2',
            'srs_2','sos_2'
            ]
ps_predict_class_names = ['t1_win']




In [16]:
def scale_features(data, col_names):
    scaled_features = {}
    for col_name in col_names:
        mean, std = data[col_name].values.mean(), data[col_name].values.std()
        scaled_features[col_name] = [mean, std]
        data.loc[:, col_name] = (data[col_name].values - mean)/std
    return scaled_features


In [17]:
scale_features(ps_game_w_team_stats, ps_feature_col_names)
ps_x = ps_game_w_team_stats[ps_feature_col_names].values
ps_y = ps_game_w_team_stats[ps_predict_class_names].values
print(type(ps_x))
split_test_size = 0.20
ps_x_train, ps_x_test, ps_y_train, ps_y_test = sklearn.model_selection.train_test_split(ps_x, ps_y, test_size=split_test_size, random_state=42)

<class 'numpy.ndarray'>


In [18]:
print("{0:0.2f}% in training set".format((len(ps_x_train)/len(ps_game_w_team_stats.index))*100))
print("{0:0.2f}% in test set".format((len(ps_x_test)/len(ps_game_w_team_stats.index))*100))

79.82% in training set
20.18% in test set


In [19]:
import sklearn.impute

#Impute with mean all 0 readings
fill_0 = sklearn.impute.SimpleImputer(missing_values=0, strategy="mean")

ps_x_train = fill_0.fit_transform(ps_x_train)
ps_x_test = fill_0.fit_transform(ps_x_test)
# TODO : impute incorrect negative values such anything other than (SOS and SRS)

In [20]:
print(len(ps_x_train[0]))

40


In [21]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
model = nn.Sequential(nn.Linear(40, 30),
                     nn.ReLU(),
                     nn.Linear(30, 30),
                     nn.ReLU(),
                     nn.Linear(30, 10),
                     nn.ReLU(),
                     nn.Linear(10, 1),
                     nn.Sigmoid())
trainset = TensorDataset(torch.from_numpy(ps_x_train).float(), torch.from_numpy(ps_y_train).float())
trainloader = DataLoader(trainset, batch_size=3, shuffle=True)
testset = TensorDataset(torch.from_numpy(ps_x_test).float(), torch.from_numpy(ps_y_test).float())
testloader = DataLoader(testset, batch_size=3, shuffle=True)
print(next(iter(testloader)))

[tensor([[-1.0816,  0.0851,  1.0144, -0.4947,  0.1144,  1.2015,  1.4245, -0.3647,
         -0.0889,  1.2608, -0.5043,  1.0103,  0.6717,  0.2003, -0.1032, -0.1998,
         -0.7121, -0.1110, -0.4200,  1.0647,  1.1001,  0.2217, -1.2754,  2.1200,
         -2.3871, -2.1503, -0.1875, -0.6372, -1.1266, -1.8976, -1.2824,  0.2317,
          0.3694,  1.0633,  0.9383,  2.0491, -0.7109, -1.3030,  0.5437,  0.8732],
        [-1.2970, -0.5933,  0.6044, -0.9893, -0.4207,  0.6199, -1.2488,  0.5748,
          1.7752,  2.5122, -0.8299, -1.0444,  0.0125, -0.8831,  0.3347, -0.4212,
         -0.3641,  1.8815,  1.6986,  0.8822,  0.4309, -2.0038, -1.8080,  0.8467,
         -0.9764, -1.6102, -1.3214, -1.2420, -0.5490, -2.0893, -1.7203, -2.0458,
         -2.4815, -1.6210,  0.2673,  0.8646, -0.3072, -0.4647,  0.5795,  0.4433],
        [-1.0816,  0.7635, -0.6064,  1.3003,  0.8983, -0.2188,  0.5937,  1.3085,
         -1.6721, -0.2411, -0.7012, -1.2169,  0.2457,  0.0977,  0.0282,  0.9441,
          0.3900,  0.5279

In [22]:
import torch.optim as optim
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [23]:
# check if CUDA is available
use_cuda = torch.cuda.is_available()
print ('Using CUDA: {}'.format(use_cuda))
if use_cuda:
    model = model.cuda()

Using CUDA: True


In [24]:
def train(n_epochs, train_loader, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    # TODO: fix target tensors always zero for some reason
    valid_loss_min = np.Inf 
    print_loss_count = 20
    cuda_refresh_count = 5
    if use_cuda:
        model = model.cuda()
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        ###################
        # train the model #
        ###################
        model.train()
        train_load_iter = iter(train_loader)
        for i in range(len(train_loader)):
            data, target = next(train_load_iter)
            # print('data {}'.format(data.shape))
            # print('target {}'.format(target))
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            optimizer.zero_grad()    
            output = model(data)
            # print('target {}'.format(target))
            # print('output {}'.format(output))
            # _, argmax = output.max(-1)
            # print('argmax', argmax)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
                
            train_loss = train_loss + ((1 / (i + 1)) * (loss.data - train_loss))
            if i % print_loss_count == 0:
                print('Epoch %d, Batch %d Loss %.6f' % (epoch, i + 1, train_loss))
            if i % cuda_refresh_count == 0:
                torch.cuda.empty_cache()
        torch.save(model.state_dict(), save_path)

    # return trained model
    return model

In [25]:
model = train(200, trainloader, model, optimizer, criterion,
                      use_cuda, 'model_nn1.pt')

Epoch 1, Batch 1 Loss 0.676714
Epoch 1, Batch 21 Loss 0.697408
Epoch 1, Batch 41 Loss 0.692889
Epoch 1, Batch 61 Loss 0.691939
Epoch 1, Batch 81 Loss 0.691744
Epoch 1, Batch 101 Loss 0.692130
Epoch 2, Batch 1 Loss 0.678610
Epoch 2, Batch 21 Loss 0.686374
Epoch 2, Batch 41 Loss 0.689130
Epoch 2, Batch 61 Loss 0.693023
Epoch 2, Batch 81 Loss 0.692332
Epoch 2, Batch 101 Loss 0.691015
Epoch 3, Batch 1 Loss 0.691336
Epoch 3, Batch 21 Loss 0.687350
Epoch 3, Batch 41 Loss 0.690566
Epoch 3, Batch 61 Loss 0.691350
Epoch 3, Batch 81 Loss 0.689946
Epoch 3, Batch 101 Loss 0.690885
Epoch 4, Batch 1 Loss 0.703332
Epoch 4, Batch 21 Loss 0.686845
Epoch 4, Batch 41 Loss 0.687674
Epoch 4, Batch 61 Loss 0.689289
Epoch 4, Batch 81 Loss 0.689807
Epoch 4, Batch 101 Loss 0.689221
Epoch 5, Batch 1 Loss 0.698609
Epoch 5, Batch 21 Loss 0.685305
Epoch 5, Batch 41 Loss 0.685491
Epoch 5, Batch 61 Loss 0.686644
Epoch 5, Batch 81 Loss 0.686567
Epoch 5, Batch 101 Loss 0.688496
Epoch 6, Batch 1 Loss 0.700044
Epoch 6, 

Epoch 42, Batch 81 Loss 0.645701
Epoch 42, Batch 101 Loss 0.644455
Epoch 43, Batch 1 Loss 0.614976
Epoch 43, Batch 21 Loss 0.629384
Epoch 43, Batch 41 Loss 0.645390
Epoch 43, Batch 61 Loss 0.644328
Epoch 43, Batch 81 Loss 0.647051
Epoch 43, Batch 101 Loss 0.645483
Epoch 44, Batch 1 Loss 0.480838
Epoch 44, Batch 21 Loss 0.646055
Epoch 44, Batch 41 Loss 0.625246
Epoch 44, Batch 61 Loss 0.634952
Epoch 44, Batch 81 Loss 0.635735
Epoch 44, Batch 101 Loss 0.638113
Epoch 45, Batch 1 Loss 0.663670
Epoch 45, Batch 21 Loss 0.650093
Epoch 45, Batch 41 Loss 0.638925
Epoch 45, Batch 61 Loss 0.635027
Epoch 45, Batch 81 Loss 0.643266
Epoch 45, Batch 101 Loss 0.640858
Epoch 46, Batch 1 Loss 0.572070
Epoch 46, Batch 21 Loss 0.646685
Epoch 46, Batch 41 Loss 0.635748
Epoch 46, Batch 61 Loss 0.634514
Epoch 46, Batch 81 Loss 0.636604
Epoch 46, Batch 101 Loss 0.633857
Epoch 47, Batch 1 Loss 0.625921
Epoch 47, Batch 21 Loss 0.628945
Epoch 47, Batch 41 Loss 0.640913
Epoch 47, Batch 61 Loss 0.637245
Epoch 47, 

Epoch 84, Batch 21 Loss 0.543963
Epoch 84, Batch 41 Loss 0.543347
Epoch 84, Batch 61 Loss 0.548694
Epoch 84, Batch 81 Loss 0.559585
Epoch 84, Batch 101 Loss 0.557273
Epoch 85, Batch 1 Loss 0.624502
Epoch 85, Batch 21 Loss 0.542435
Epoch 85, Batch 41 Loss 0.557760
Epoch 85, Batch 61 Loss 0.537415
Epoch 85, Batch 81 Loss 0.558821
Epoch 85, Batch 101 Loss 0.558982
Epoch 86, Batch 1 Loss 0.488531
Epoch 86, Batch 21 Loss 0.587742
Epoch 86, Batch 41 Loss 0.572942
Epoch 86, Batch 61 Loss 0.547589
Epoch 86, Batch 81 Loss 0.539080
Epoch 86, Batch 101 Loss 0.536455
Epoch 87, Batch 1 Loss 0.598504
Epoch 87, Batch 21 Loss 0.608040
Epoch 87, Batch 41 Loss 0.569166
Epoch 87, Batch 61 Loss 0.563444
Epoch 87, Batch 81 Loss 0.543986
Epoch 87, Batch 101 Loss 0.558097
Epoch 88, Batch 1 Loss 1.104342
Epoch 88, Batch 21 Loss 0.578391
Epoch 88, Batch 41 Loss 0.544198
Epoch 88, Batch 61 Loss 0.544738
Epoch 88, Batch 81 Loss 0.548903
Epoch 88, Batch 101 Loss 0.545020
Epoch 89, Batch 1 Loss 0.246702
Epoch 89, 

Epoch 125, Batch 1 Loss 1.098436
Epoch 125, Batch 21 Loss 0.571507
Epoch 125, Batch 41 Loss 0.503677
Epoch 125, Batch 61 Loss 0.500087
Epoch 125, Batch 81 Loss 0.520796
Epoch 125, Batch 101 Loss 0.511377
Epoch 126, Batch 1 Loss 0.647299
Epoch 126, Batch 21 Loss 0.617444
Epoch 126, Batch 41 Loss 0.521182
Epoch 126, Batch 61 Loss 0.510287
Epoch 126, Batch 81 Loss 0.511193
Epoch 126, Batch 101 Loss 0.499713
Epoch 127, Batch 1 Loss 1.294948
Epoch 127, Batch 21 Loss 0.487475
Epoch 127, Batch 41 Loss 0.486822
Epoch 127, Batch 61 Loss 0.473134
Epoch 127, Batch 81 Loss 0.483176
Epoch 127, Batch 101 Loss 0.482579
Epoch 128, Batch 1 Loss 0.167551
Epoch 128, Batch 21 Loss 0.514376
Epoch 128, Batch 41 Loss 0.525089
Epoch 128, Batch 61 Loss 0.552380
Epoch 128, Batch 81 Loss 0.524001
Epoch 128, Batch 101 Loss 0.503069
Epoch 129, Batch 1 Loss 0.307353
Epoch 129, Batch 21 Loss 0.495351
Epoch 129, Batch 41 Loss 0.469860
Epoch 129, Batch 61 Loss 0.458770
Epoch 129, Batch 81 Loss 0.470140
Epoch 129, Batc

Epoch 165, Batch 21 Loss 0.381031
Epoch 165, Batch 41 Loss 0.403324
Epoch 165, Batch 61 Loss 0.420352
Epoch 165, Batch 81 Loss 0.408552
Epoch 165, Batch 101 Loss 0.416907
Epoch 166, Batch 1 Loss 0.787279
Epoch 166, Batch 21 Loss 0.365914
Epoch 166, Batch 41 Loss 0.383552
Epoch 166, Batch 61 Loss 0.441046
Epoch 166, Batch 81 Loss 0.427785
Epoch 166, Batch 101 Loss 0.428736
Epoch 167, Batch 1 Loss 0.348670
Epoch 167, Batch 21 Loss 0.484251
Epoch 167, Batch 41 Loss 0.463645
Epoch 167, Batch 61 Loss 0.458979
Epoch 167, Batch 81 Loss 0.451365
Epoch 167, Batch 101 Loss 0.441387
Epoch 168, Batch 1 Loss 0.333101
Epoch 168, Batch 21 Loss 0.464985
Epoch 168, Batch 41 Loss 0.430875
Epoch 168, Batch 61 Loss 0.409984
Epoch 168, Batch 81 Loss 0.424432
Epoch 168, Batch 101 Loss 0.419125
Epoch 169, Batch 1 Loss 0.118342
Epoch 169, Batch 21 Loss 0.412518
Epoch 169, Batch 41 Loss 0.417471
Epoch 169, Batch 61 Loss 0.431836
Epoch 169, Batch 81 Loss 0.407606
Epoch 169, Batch 101 Loss 0.422759
Epoch 170, Ba

In [26]:
def test(loader, model, criterion, use_cuda, num_classes = 2):
    if use_cuda:
        model = model.cuda()
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    model.eval()
    load_iter = iter(loader)
    for i in range(len(loader)):
        data, target = next(load_iter)
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (i + 1)) * (loss.data - test_loss))
        # compare predictions to true label
        for j, tensor in enumerate(output):
            if (tensor.item() > .5 and target[j] == 1) or (tensor.item() <= .5 and target[j] == 0):
                correct += 1
        total += data.size(0)
       
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [27]:
test(testloader, model, criterion, use_cuda)

Test Loss: 0.446631


Test Accuracy: 79% (71/89)
